<a href="https://colab.research.google.com/github/BraveheartNL/ISIMI-2021/blob/Lennart_Project/Project/segment_nodule.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from os import walk

import SimpleITK
import SimpleITK as sitk
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
import matplotlib.colors as colors
from matplotlib import cm
%matplotlib inline 
import numpy as np
matplotlib.rcParams['figure.figsize'] = (20, 12)
import pandas as pd
from pandas import DataFrame
from medpy.io import load, save
import random
import PIL
from PIL import Image
import scipy.ndimage
import json
import math

In [2]:
config = json.load(open('config.json'))
data_folder_path = config["data_folder_path"]
print(data_folder_path)
ct_folder = f'{data_folder_path}/luna16_nodules/nodule_patches/'
mask_folder = f'{data_folder_path}/luna16_nodules/segmentation/'
original_xray_folder = f"{data_folder_path}/original/"
xray_folder = f"{data_folder_path}/generation_data/preprocessed_images/"
lung_segment_folder = f"{data_folder_path}/generation_data/lung_segmentations/"
cropped_nodules_folder = f"{data_folder_path}/valid_nodules/"
results_folder = f"{data_folder_path}/extended_training_data/images/"
evaluation_data_folder = f"{data_folder_path}/evaluation_data/"
training_data_folder = f"{data_folder_path}/preprocessed/"

_, _, xray_names = next(walk(xray_folder))
_, _, ct_names = next(walk(ct_folder))
_, _, mask_names = next(walk(mask_folder))
_, _, cropped_nodule_names = next(walk(cropped_nodules_folder))
_, _, lung_segment_names = next(walk(lung_segment_folder))
_, _, training_data_names = next(walk(training_data_folder))

D:\TrainingData\ISMI Final Project


In [3]:
def resample_nodule(ct_data, xray_spacing):
    resize_factor = [1,1,1]/xray_spacing
    new_real_shape = ct_data.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / ct_data.shape
    new_spacing = [1,1,1] / real_resize_factor
    resampled_nodule = scipy.ndimage.interpolation.zoom(ct_data, real_resize_factor, mode='nearest')
    return resampled_nodule

def segment_nodule(ct, mask):
    background = np.full(ct.shape, np.min(ct))
    segmented_nodule = np.where(mask == 0, background, ct)
    return segmented_nodule

#Code from the baseline shared by Ecem
def project_ct(X_ct, p_lambda = 0.85):
    '''
    Generate 2D digitally reconstructed radiographs from CT scan. (DRR, fake CXR, simulated CXR)
    X_ct: CT scan
    p-lambda:  β controls the boosting of X-ray absorption as the tissue density increases.
    We have chosen β=0.85 for our experiments after performing a visual comparison with real chest X-rays.
    author: Ecem Sogancioglu
    '''
    X_ct[X_ct > 400] = 400
    X_ct[X_ct < -500] = -500
    X_ct[X_ct < -1024] = -1024
    X_ct += 1024
    # 1424 524 698.748232
    X_ct = X_ct/1000.0
    X_ct *= p_lambda
    X_ct[X_ct > 1] = 1
    #1.0 0.4454 0.5866707652
    X_ct_2d = np.mean(np.exp(X_ct), axis=1)
    return X_ct_2d

def crop_nodule(ct_2d):
    outlines = np.where(ct_2d != np.min(ct_2d))
    x_min = np.min(outlines[0])
    x_max = np.max(outlines[0])
    y_min = np.min(outlines[1])
    y_max = np.max(outlines[1])

    cropped_patch = ct_2d[x_min:x_max+1, y_min:y_max+1]
      
    return cropped_patch

def normalize_x_ray(cxr):
    return ((cxr - cxr.min()) * (1/(cxr.max() - cxr.min()) * 255)).astype('uint8')
    

In [4]:
def preprocess_nodule(xray_header, ct, mask):
    #Resample the ct and the mask based on Xray spacing

    xray_spacing = xray_header.get_voxel_spacing()
    resampled_ct = resample_nodule(ct, np.array([xray_spacing[0],xray_spacing[1],xray_spacing[0]]))
    resampled_mask = resample_nodule(mask, np.array([xray_spacing[0],xray_spacing[1],xray_spacing[0]]))

    #Segment the nodule using the mask
    segmented_ct = segment_nodule(resampled_ct, resampled_mask)
    
    #Project the segmented CT data from 3d to 2d
    ct_2d = project_ct(segmented_ct)

    return ct_2d

def normalize_nodule(xray_arr, cropped_nodule, location):
    x = location[0]
    y = location[1]
    ct_2d = cropped_nodule

    x_min, x_max = ct_2d.shape[0]//2, ct_2d.shape[0]//2
    y_min, y_max = ct_2d.shape[1]//2, ct_2d.shape[1]//2

    if ct_2d.shape[0] % 2 != 0:
        x_max+=1
    if ct_2d.shape[1] % 2 != 0:
        y_max+=1

    #normalize projected ct
    n_max = np.max(xray_arr[ x-x_min:x+x_max, y-y_min:y+y_max ])
    n_min = np.min(xray_arr[ x-x_min:x+x_max, y-y_min:y+y_max ])
    ct_2d = ((ct_2d-ct_2d.min())*(1/(ct_2d.max() - ct_2d.min()) * (n_max - n_min))+n_min).astype('uint8')
    return ct_2d

In [5]:
#Superimposition section
#spot on the CXR to locate the nodule
def normalization(x):
    x = x.copy()
    x = (x - x.min()) / (x.max() - x.min())
    return x


def mean_superimpose(location: tuple, nodule: np.array, xray: np.array) -> np.array:
    x = location[0]
    y = location[1]
    x_min, x_max = nodule.shape[0]//2, nodule.shape[0]//2
    y_min, y_max = nodule.shape[1]//2, nodule.shape[1]//2

    if nodule.shape[0] % 2 != 0:
        x_max+=1
    if nodule.shape[1] % 2 != 0:
        y_max+=1
    xray_patch = xray[x-x_min:x+x_max,y-y_min:y+y_max]
    valued_indices = (nodule > 1.1 * nodule.min())
    # blend cxr patch and nodule patch
    nodule[valued_indices] = np.mean(np.array([xray_patch[valued_indices],nodule[valued_indices]]), axis=0)
    nodule[~valued_indices] = xray_patch[~valued_indices]

    xray[x-x_min:x+x_max,y-y_min:y+y_max] = nodule
    return xray

'''
def litjens_mean_superimpose(location: tuple, nodule: np.array, xray: np.array) -> np.array:
    x = location[0]
    y = location[1]
    x_min, x_max = nodule.shape[0]//2, nodule.shape[0]//2
    y_min, y_max = nodule.shape[1]//2, nodule.shape[1]//2

    if nodule.shape[0] % 2 != 0:
        x_max+=1
    if nodule.shape[1] % 2 != 0:
        y_max+=1
    xray_patch = xray[x-x_min:x+x_max,y-y_min:y+y_max]
    valued_indices = nodule != (1.0 * nodule.min())
    nodule[valued_indices] = np.mean(np.array([xray_patch[valued_indices],nodule[valued_indices]]), axis=0)
    nodule[~valued_indices] = xray_patch[~valued_indices]

    xray[x-x_min:x+x_max,y-y_min:y+y_max] = nodule

    C = xray_patch.max() - xray_patch.min()
    D = np.sqrt(nodule.shape[0]**2 + nodule.shape[1]**2)
    for i in range(nodule.shape[0]):
        for j in range(nodule.shape[1]):
            r = np.sqrt((nodule.shape[0]//2-i)**2 + (nodule.shape[1]//2-j)**2)
            c = C*( 4*r**4/D**4 - 4.2*r**2/D**2 + 1)
            xray[x - nodule.shape[0]//2+i, y-nodule.shape[1]//2+j]+=nodule[i,j]*c
    return xray
'''

'\ndef litjens_mean_superimpose(location: tuple, nodule: np.array, xray: np.array) -> np.array:\n    x = location[0]\n    y = location[1]\n    x_min, x_max = nodule.shape[0]//2, nodule.shape[0]//2\n    y_min, y_max = nodule.shape[1]//2, nodule.shape[1]//2\n\n    if nodule.shape[0] % 2 != 0:\n        x_max+=1\n    if nodule.shape[1] % 2 != 0:\n        y_max+=1\n    xray_patch = xray[x-x_min:x+x_max,y-y_min:y+y_max]\n    valued_indices = nodule != (1.0 * nodule.min())\n    nodule[valued_indices] = np.mean(np.array([xray_patch[valued_indices],nodule[valued_indices]]), axis=0)\n    nodule[~valued_indices] = xray_patch[~valued_indices]\n\n    xray[x-x_min:x+x_max,y-y_min:y+y_max] = nodule\n\n    C = xray_patch.max() - xray_patch.min()\n    D = np.sqrt(nodule.shape[0]**2 + nodule.shape[1]**2)\n    for i in range(nodule.shape[0]):\n        for j in range(nodule.shape[1]):\n            r = np.sqrt((nodule.shape[0]//2-i)**2 + (nodule.shape[1]//2-j)**2)\n            c = C*( 4*r**4/D**4 - 4.2*r**

In [ ]:
gen_metadata = {
        'height':    [],
        'original_name': [],
        'width':     [],
        'x'    :     [],
        'y'    :     [],
        'dataset':   [],
        'img_name':  []
        }

for i in range(1134):
    #pick random xray and nodule
    random_xray_index = np.random.randint(0, len(xray_names))
    random_cropped_nodule_index = np.random.randint(0, len(cropped_nodule_names))
    
    # normalize cxr
    xray, xray_header = load(os.path.join(xray_folder, xray_names[random_xray_index]))
    xray_mask, xray_mask_header = load(os.path.join(lung_segment_folder, lung_segment_names[random_xray_index]))
    n_xray = normalize_x_ray(xray)
    
    #choose location to place nodule
    valid_y, valid_x = np.where(xray_mask)
    location = np.random.randint(len(valid_x))
    x, y = [valid_x[location], valid_y[location]]

    # use valid_nodules.
    random_cropped_nodule_file = cropped_nodule_names[random_cropped_nodule_index]
    random_cropped_nodule = load(os.path.join(cropped_nodules_folder, random_cropped_nodule_file))[0]
    
    normalized_nodule = normalize_nodule(xray_arr=n_xray,
                                     cropped_nodule=random_cropped_nodule,
                                     location=(y, x))

    result = mean_superimpose((y, x), normalized_nodule, n_xray)

    save(result, f"{results_folder}{1135+i}.png")
    
    #update metadata
    gen_metadata['height'].append(random_cropped_nodule.shape[0])
    gen_metadata['original_name'].append('-')
    gen_metadata['width'].append(random_cropped_nodule.shape[1])
    gen_metadata['x'].append(x)
    gen_metadata['y'].append(y)
    gen_metadata['dataset'].append('-')
    gen_metadata['img_name'].append(f"{1135+i}.png")
    
    if i%50 == 0:
        print(i)


In [ ]:
gen_metadata_df = DataFrame(gen_metadata)
gen_metadata_df.info()

In [8]:
og_metadata_df = pd.read_csv('./Data/training_data/metadata.csv')
og_metadata_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2106 entries, 0 to 2105
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   height         2106 non-null   int64 
 1   original_name  2106 non-null   object
 2   width          2106 non-null   int64 
 3   x              2106 non-null   int64 
 4   y              2106 non-null   int64 
 5   dataset        2106 non-null   object
 6   img_name       2106 non-null   object
dtypes: int64(4), object(3)
memory usage: 115.3+ KB


In [9]:
complete_metadata_df = pd.concat([og_metadata_df, gen_metadata_df])
complete_metadata_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3240 entries, 0 to 1133
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   height         3240 non-null   int64 
 1   original_name  3240 non-null   object
 2   width          3240 non-null   int64 
 3   x              3240 non-null   int64 
 4   y              3240 non-null   int64 
 5   dataset        3240 non-null   object
 6   img_name       3240 non-null   object
dtypes: int64(4), object(3)
memory usage: 202.5+ KB


In [10]:
complete_metadata_df.to_csv(f'{data_folder_path}/extended_training_data/metadata.csv', index=False, mode='w+')

In [33]:
def generate_data_set(names, all_metadata, output_file, num_files=100):
    """
    generate dataset and

    :param names: all names of files (.mha files)
    :param all_metadata: metadata of names files
    :param output_file: outputs normalized images of num_files random records from names, and metadata file
    :param num_files: num files saved to output_file and represented in meta datafile
    :return: None
    """
    dataset_metadata = pd.DataFrame(columns=all_metadata.columns)
    dataset_names = names[np.random.randint(0, len(names), size=num_files)]
    for name in dataset_names:
        img = load(os.path.join(training_data_folder, name))[0]
        img = normalize_x_ray(img)
        save(img, os.path.join(evaluation_data_folder, name.split(".")[0]+".png"))
        if name in list(all_metadata['img_name']):
            records = all_metadata[all_metadata['img_name'] == name]
            records['img_name'] = records['img_name'].iloc[0].split('.')[0]+".png"
            dataset_metadata = dataset_metadata.append(records)

    dataset_metadata.to_csv(output_file)

In [34]:
generate_data_set(np.array(training_data_names),
                  pd.read_csv(f"{data_folder_path}/metadata_preprocessed.csv"),
                  os.path.join(evaluation_data_folder, "eval_metadata.csv"))


<ipython-input-33-0e5782eacf9b>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  records['img_name'] = records['img_name'].iloc[0].split('.')[0]+".png"
